In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import wikipedia as wp
from pypostalcode import PostalCodeDatabase

In [2]:
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[0]
df.to_csv('toronto.csv',header=1,index=False)

In [3]:
df = pd.read_csv('toronto.csv')
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [4]:
df['Borough'].replace('Not assigned', np.nan, inplace=True)
df.head()

,Postal code,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [5]:
df.dropna(subset=['Borough'], inplace=True)
df['Neighborhood'].replace('Not assigned', df['Borough'], inplace=True)
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [6]:
df.shape

(103, 3)

In [7]:
p = PostalCodeDatabase()
df['Postal code'].replace('M7R',np.nan,inplace=True)
df.dropna(subset=['Postal code'], inplace=True)
#'M7R' not registered in pypostalcode

In [8]:
c=df['Postal code']
e={}
f={}
for i in range(0,102):
      e[c.iloc[i]] = (p[c.iloc[i]].longitude)
for i in range(0,102):
      f[c.iloc[i]] = (p[c.iloc[i]].latitude)
ds = pd.DataFrame()
ds['Postal code'] = e.keys()
ds['Longitude'] = e.values()
ds['Latitude']=f.values()
df = pd.merge(df, ds, on='Postal code', how='outer')
df.head()

,Postal code,Borough,Neighborhood,Longitude,Latitude
0,M3A,North York,Parkwoods,-79.3280,43.7358
1,M4A,North York,Victoria Village,-79.3037,43.7159
2,M5A,Downtown Toronto,Regent Park / Harbourfront,-79.3505,43.6369
3,M6A,North York,Lawrence Manor / Lawrence Heights,-79.4300,43.7193
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,-79.3889,43.6641


In [9]:
dt = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
dt.head()

,Postal code,Borough,Neighborhood,Longitude,Latitude
0,M5A,Downtown Toronto,Regent Park / Harbourfront,-79.3505,43.6369
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,-79.3889,43.6641
2,M5B,Downtown Toronto,"Garden District, Ryerson",-79.3796,43.6543
3,M5C,Downtown Toronto,St. James Town,-79.5318,43.6870
4,M4E,East Toronto,The Beaches,-79.2960,43.6675


In [10]:
toronto_latitude = 43.6532; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(dt['Latitude'], dt['Longitude'], dt['Borough'], dt['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

In [11]:
CLIENT_ID = 'PQ0K0E5GZCDP4QJTKM4ZRAU0TZPAG1HPA0X55NY3DPQAGN33'
CLIENT_SECRET = 'YDKJ215ZQB42YLTCRS5YZCNVFYHRIGVTMBD4MPRK3IN0ZEXP' 
VERSION = '20180605'
LIMIT=100 
radius=500

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
downtown_venues = getNearbyVenues(names=dt['Neighborhood'],
                                   latitudes=dt['Latitude'],
                                   longitudes=dt['Longitude']
                                  )

Regent Park / Harbourfront
Queen's Park / Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
The Danforth West / Riverdale
Toronto Dominion Centre / Design Exchange
Brockton / Parkdale Village / Exhibition Place
India Bazaar / The Beaches West
Commerce Court / Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park / The Junction South
North Toronto West
The Annex / North Midtown / Yorkville
Parkdale / Roncesvalles
Davisville
University of Toronto / Harbord
Runnymede / Swansea
Moore Park / Summerhill East
Kensington Market / Chinatown / Grange Park
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport
Roseda

In [14]:
downtown_venues.shape

(2016, 7)

In [15]:
downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,27,27,27,27,27,27
Brockton / Parkdale Village / Exhibition Place,33,33,33,33,33,33
Business reply mail Processing CentrE,17,17,17,17,17,17
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,41,41,41,41,41,41
Central Bay Street,75,75,75,75,75,75
Christie,61,61,61,61,61,61
Church and Wellesley,69,69,69,69,69,69
Commerce Court / Victoria Hotel,100,100,100,100,100,100
Davisville,21,21,21,21,21,21


In [16]:
downtown_venues['Venue Category'].unique()[:100]

array(['Dog Run', 'Pool', 'Soccer Stadium', 'Golf Driving Range',
       'Boat or Ferry', 'Harbor / Marina', 'Beach', 'Park',
       'Sushi Restaurant', 'Beer Bar', 'Bubble Tea Shop', 'Gastropub',
       'Mexican Restaurant', 'Distribution Center', 'Dance Studio',
       'Ramen Restaurant', 'Diner', 'Creperie', 'Burrito Place',
       'Arts & Crafts Store', 'Italian Restaurant', 'Burger Joint',
       'Hobby Shop', 'Café', 'Martial Arts Dojo', 'Indian Restaurant',
       'College Theater', 'Coffee Shop', 'Theater', 'Gym',
       'Clothing Store', 'Cosmetics Shop', 'Bookstore', 'Shopping Mall',
       'Plaza', 'Electronics Store', 'Furniture / Home Store',
       'Music Venue', 'Restaurant', 'Bakery', 'Breakfast Spot',
       'American Restaurant', 'Fast Food Restaurant', 'Department Store',
       'Comic Shop', 'Tea Room', 'Lingerie Store', 'Hotel',
       'Monument / Landmark', 'Office', 'Sporting Goods Shop',
       'New American Restaurant', 'Miscellaneous Shop', 'Neighborhood',
   

In [17]:
"Indian Restaurant" in downtown_venues['Venue Category'].unique()

True

In [18]:
onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")
onehot['Neighborhood'] = downtown_venues['Neighborhood'] # add neighborhood column back to dataframe
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])# move neighborhood column to the first column
onehot = onehot[fixed_columns]
print(onehot.shape)
onehot.head()

(2016, 250)


,Yoga Studio,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,Hobby Shop,Hong Kong Restaurant,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tree,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [19]:
grouped = onehot.groupby(["Neighborhood"]).mean().reset_index()
grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,Hobby Shop,Hong Kong Restaurant,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tree,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store
0,Berczy Park,0.000000,0.0,0.000000,0.074074,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.037037,0.000000,0.000000,0.0,0.0,0.037037,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.074074,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.037037,0.037037,0.0,0.0,0.037037,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.037037,0.0,0.037037,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.037037,0.00000,0.0,0.00000,0.

In [20]:
print(len(grouped[grouped["Indian Restaurant"] > 0]))
indian = grouped[["Neighborhood","Indian Restaurant"]]
indian.head(10)

9


,Neighborhood,Indian Restaurant
0,Berczy Park,0.037037
1,Brockton / Parkdale Village / Exhibition Place,0.000000
2,Business reply mail Processing CentrE,0.000000
3,CN Tower / King and Spadina / Railway Lands / ...,0.000000
4,Central Bay Street,0.000000
5,Christie,0.000000
6,Church and Wellesley,0.000000
7,Commerce Court / Victoria Hotel,0.000000
8,Davisville,0.047619
9,Davisville North,0.014286


In [21]:
clusters = 5
clustering = indian.drop(["Neighborhood"], 1)
# run k-means clustering
kmeans = KMeans(n_clusters=clusters, random_state=1)
kmeans.fit_transform(clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

array([1, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32)

In [22]:
indian["Cluster Labels"] = kmeans.labels_

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [23]:
indian = indian.join(downtown_venues.set_index("Neighborhood"), on="Neighborhood")
#print(indian.shape)
indian

,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.037037,1,43.6390,-79.4499,Budapest Park,43.636963,-79.449103,Park
0,Berczy Park,0.037037,1,43.6390,-79.4499,Sunnyside Beach,43.637466,-79.453235,Beach
0,Berczy Park,0.037037,1,43.6390,-79.4499,Easy Restaurant,43.638702,-79.445843,Breakfast Spot
0,Berczy Park,0.037037,1,43.6390,-79.4499,South Indian Dosa Mahal,43.639184,-79.446109,Indian Restaurant
0,Berczy Park,0.037037,1,43.6390,-79.4499,Village Juicery,43.642064,-79.447367,Juice Bar
0,Berczy Park,0.037037,1,43.6390,-79.4499,Lambretta Pizzeria & Wine Bar,43.641683,-79.447209,Pizza Place
0,Berczy Park,0.037037,1,43.6390,-79.4499,The Simple Kitchen,43.641316,-79.447109,Restaurant
0,Berczy Park,0.037037,1,43.6390,-79.4499,Cherry Bomb,43.641477,-79.447136,Coffee Shop
0,Berczy Park,0.037037,1,43.6390,-79.4499,North Shore Pie Company,43.640336,-79.446729,Pie Shop
0,Berczy Park,0.037037,1,43.6390,-79.4499,Benna's Bakery & Deli,43.643170,-79.447849,Bakery


In [24]:
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude],zoom_start=14)

# set color scheme for the clusters


# add markers to the map
markers_colors={}
markers_colors[0] = 'red'
markers_colors[1] = 'blue'
markers_colors[2] = 'green'
markers_colors[3] = 'yellow'
markers_colors[4] = 'cyan'
markers_colors[5] = 'black'
for lat, lon, cluster in zip(indian['Neighborhood Latitude'], indian['Neighborhood Longitude'], indian['Cluster Labels']):
    
    
    folium.CircleMarker(
        [lat, lon],
        radius=5,
       
        color =markers_colors[cluster],
        fill_color=markers_colors[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [25]:
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude],zoom_start=14)

# set color scheme for the clusters


# add markers to the map
markers_colors={}
markers_colors[0] = 'red'
markers_colors[1] = 'blue'
markers_colors[2] = 'green'
markers_colors[3] = 'yellow'
markers_colors[4] = 'cyan'
markers_colors[5] = 'black'
for lat, lon, cluster in zip(indian['Neighborhood Latitude'], indian['Neighborhood Longitude'], indian['Cluster Labels']):
    
    
    folium.CircleMarker(
        [lat, lon],
        radius=5,
       
        color =markers_colors[cluster],
        fill_color=markers_colors[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [26]:
indian.loc[(indian['Cluster Labels'] ==0) & (indian['Venue Category'] == 'Indian Restaurant') ]

,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


In [27]:
indian.loc[(indian['Cluster Labels'] ==1) & (indian['Venue Category'] == 'Indian Restaurant') ]

,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.037037,1,43.6390,-79.4499,South Indian Dosa Mahal,43.639184,-79.446109,Indian Restaurant
22,Parkdale / Roncesvalles,0.040816,1,43.6403,-79.4374,Mother India,43.640718,-79.436259,Indian Restaurant
22,Parkdale / Roncesvalles,0.040816,1,43.6403,-79.4374,Om Restaurant,43.640337,-79.437811,Indian Restaurant
23,Queen's Park / Ontario Provincial Government,0.040000,1,43.6641,-79.3889,Kothur Indian Cuisine,43.667872,-79.385659,Indian Restaurant


In [28]:
indian.loc[(indian['Cluster Labels'] ==2) & (indian['Venue Category'] == 'Indian Restaurant') ]

,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
12,Forest Hill North & West,0.024390,2,43.6981,-79.3987,Flaming Stove,43.698273,-79.395831,Indian Restaurant
26,Rosedale,0.020000,2,43.6699,-79.3887,Utsav Indian Cuisine,43.671154,-79.390802,Indian Restaurant
26,Rosedale,0.020000,2,43.6699,-79.3887,Kothur Indian Cuisine,43.667872,-79.385659,Indian Restaurant
30,St. James Town / Cabbagetown,0.018868,2,43.6647,-79.3695,Butter Chicken Factory,43.667072,-79.369184,Indian Restaurant
34,The Annex / North Midtown / Yorkville,0.020000,2,43.6705,-79.3901,Utsav Indian Cuisine,43.671154,-79.390802,Indian Restaurant
34,The Annex / North Midtown / Yorkville,0.020000,2,43.6705,-79.3901,Kothur Indian Cuisine,43.667872,-79.385659,Indian Restaurant


# Conclusion
### Most of the Indian restaurants are around cluster label 2 which is Central Bay Street, Church and Wellesley, Berczy Park, Union Station, and Richmond. This would imply higher competition if restaurant is set up it would be tough to gain sales.

### And lowest in Cluster 0 areas which are in North Toronto West and Parkade areas.Also, there are good opportunities to open near St James Town, Cabbagetown, Toronto Centre, Regent Park, Chruch Wellesley Village.
### Looking at nearby venues it seems cluster 1 might be a good location as there are not a lot of Indian restaurants in these areas. Therefore, this project recommends the entrepreneur to open an authentic Indian restaurant in these locations.